# NLP Unit 7

## Part 1: WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS
https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py

**See tutorial on Website -- Here we provide additional content / Exercise results**


In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Exercise: Computing Word Embeddings: Continuous Bag-of-Words

In [2]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

def create_context_data(raw_text):

    data = []
    for i in range(2, len(raw_text) - 2):
        context = [raw_text[i - 2], raw_text[i - 1],
                   raw_text[i + 1], raw_text[i + 2]]
        target = raw_text[i]
        data.append((context, target))

    print(data[:5])
    print("\nNumber of training examples:", len(data))
    print("vocab_size", vocab_size)

    return data

data = create_context_data(raw_text)


[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]

Number of training examples: 58
vocab_size 49


In [3]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

tensor([39, 19, 32, 30])

**initial skeleton code:**

In [4]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

In [5]:
EMBEDDING_DIM = 50
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size *  2 * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

In [6]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(20):
    total_loss = 0
    for context, target in data:
                
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        #print(context, target)
        context_idxs = make_context_vector(context, word_to_ix)  # example
        #print(context_ids)
        
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[225.63277626037598, 222.4949815273285, 219.39596486091614, 216.32562494277954, 213.27869820594788, 210.24801349639893, 207.2434663772583, 204.24631023406982, 201.26555013656616, 198.29729437828064, 195.3220534324646, 192.35428929328918, 189.38395929336548, 186.41633296012878, 183.43802070617676, 180.4414987564087, 177.45058417320251, 174.42366194725037, 171.38644242286682, 168.33168530464172]


<font color="red">
    
### Do some predictions to see if works somehow :)

In [7]:
test_sentence = "We study the idea of computational processes".split()
test_data = create_context_data(test_sentence)
print(test_data)

## we need an idx-to-word
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for word, i in word_to_ix.items()}
#print(ix_to_word)

with torch.no_grad():
    for context, target in test_data:
        context_idxs = make_context_vector(context, word_to_ix)  # example
        log_probs = model(context_idxs)
        
        # argmax give the index of the entry with the largest value
        pred_idx = log_probs.argmax().item()
        print(context, ix_to_word[pred_idx])
        

[(['We', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'computational'], 'idea'), (['the', 'idea', 'computational', 'processes'], 'of')]

Number of training examples: 3
vocab_size 49
[(['We', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'computational'], 'idea'), (['the', 'idea', 'computational', 'processes'], 'of')]
['We', 'study', 'idea', 'of'] the
['study', 'the', 'of', 'computational'] of
['the', 'idea', 'computational', 'processes'] of


In [8]:
### ok see it doesn't make too much sense yet, the training data is too small

# Part 4: LSTM: SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py


In [9]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [10]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [11]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print("\n")
    print(tag_scores)


tensor([0, 1, 2, 3, 4])
tensor([[-0.8667, -1.3783, -1.1158],
        [-0.9286, -1.2928, -1.1075],
        [-0.9748, -1.3900, -0.9844],
        [-0.9014, -1.4633, -1.0147],
        [-0.8908, -1.3968, -1.0721]])


tensor([[-0.0222, -4.6487, -4.3935],
        [-5.9344, -0.0126, -4.6154],
        [-3.4598, -3.7630, -0.0562],
        [-0.0593, -4.4432, -3.0826],
        [-4.0660, -0.0221, -5.3529]])


### Exercise: Extend with Char-Level features!

In [12]:
EMBEDDING_DIM = 3 

# 0.) create a list of words in the training texts

sentences = [" ".join(sentence) for sentence, _ in training_data]

print(sentences)

# create the input and target sequences
inputs  = [s[:-1] for s in sentences]
targets = [s[1:] for s in sentences]

print(inputs)
print("Targets", targets)

# 1.) create char_to_ix
char_to_ix = {}
for sentence in sentences:
    for c in sentence:
        if c not in char_to_ix:
            char_to_ix[c] = len(char_to_ix)

print(char_to_ix)



vocab_size = len(char_to_ix)



['The dog ate the apple', 'Everybody read that book']
['The dog ate the appl', 'Everybody read that boo']
Targets ['he dog ate the apple', 'verybody read that book']
{'T': 0, 'h': 1, 'e': 2, ' ': 3, 'd': 4, 'o': 5, 'g': 6, 'a': 7, 't': 8, 'p': 9, 'l': 10, 'E': 11, 'v': 12, 'r': 13, 'y': 14, 'b': 15, 'k': 16}


In [13]:


class CharLSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(CharLSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2char = nn.Linear(hidden_dim, vocab_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        char_space = self.hidden2char(lstm_out.view(len(sentence), -1))
        char_scores = F.log_softmax(char_space, dim=1)
        return char_scores

In [14]:

model = CharLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(char_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

print(inputs)
print("TTT", targets)

print(char_to_ix)

with torch.no_grad():
    inputs_in = prepare_sequence(inputs[0], char_to_ix)
    print(inputs_in)
    char_scores = model(inputs_in)

                 

for epoch in range(2000):  # again, normally you would NOT do 300 epochs, it is toy data
    for i in range(len(inputs)):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sent_in = prepare_sequence(inputs[i], char_to_ix)
        targets_in = prepare_sequence(targets[i], char_to_ix)

        # Step 3. Run our forward pass.
        char_scores = model(sent_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(char_scores, targets_in)
        #print(loss)
        loss.backward()
        optimizer.step()
        
        
        if epoch%50 == 0:
        #print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
            print("Loss: {:.4f}".format(loss.item()))

# See what the scores are after training

with torch.no_grad():
    inputs_in = prepare_sequence(inputs[0], char_to_ix)
    print(inputs_in)
    char_scores = model(inputs_in)
    
    print("\n")
    print(char_scores)

['The dog ate the appl', 'Everybody read that boo']
TTT ['he dog ate the apple', 'verybody read that book']
{'T': 0, 'h': 1, 'e': 2, ' ': 3, 'd': 4, 'o': 5, 'g': 6, 'a': 7, 't': 8, 'p': 9, 'l': 10, 'E': 11, 'v': 12, 'r': 13, 'y': 14, 'b': 15, 'k': 16}
tensor([ 0,  1,  2,  3,  4,  5,  6,  3,  7,  8,  2,  3,  8,  1,  2,  3,  7,  9,
         9, 10])
Loss: 2.8664
Loss: 2.8985
Loss: 2.4864
Loss: 2.6731
Loss: 2.3867
Loss: 2.6116
Loss: 2.3293
Loss: 2.5545
Loss: 2.2521
Loss: 2.4790
Loss: 2.1230
Loss: 2.3779
Loss: 1.9631
Loss: 2.2354
Loss: 1.8024
Loss: 2.0601
Loss: 1.6451
Loss: 1.8818
Loss: 1.4764
Loss: 1.7065
Loss: 1.3133
Loss: 1.5217
Loss: 1.1706
Loss: 1.3549
Loss: 1.0379
Loss: 1.2176
Loss: 0.9172
Loss: 1.1011
Loss: 0.8114
Loss: 1.0052
Loss: 0.7239
Loss: 0.9162
Loss: 0.6461
Loss: 0.8249
Loss: 0.5725
Loss: 0.7426
Loss: 0.5149
Loss: 0.6867
Loss: 0.4767
Loss: 0.6418
Loss: 0.4234
Loss: 0.5963
Loss: 0.4026
Loss: 0.5661
Loss: 0.3627
Loss: 0.5304
Loss: 0.3446
Loss: 0.4992
Loss: 0.3132
Loss: 0.4811
L